In [ ]:
cd /content/drive/MyDrive/nervlsp2018/VLSP2018-NER-train-Jan14

/content/drive/MyDrive/nervlsp2018/VLSP2018-NER-train-Jan14


In [ ]:
import pandas as pd

df = pd.read_csv('combined_csv_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314525 entries, 0 to 314524
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   sentence_idx  314525 non-null  int64 
 1   word          314523 non-null  object
 2   pos           314525 non-null  object
 3   tag           314525 non-null  object
dtypes: int64(1), object(3)
memory usage: 9.6+ MB


In [ ]:
df

,sentence_idx,word,pos,tag
0,1,Tiếp,V,O
1,1,tục,V,O
2,1,xảy,V,O
3,1,ra,V,O
4,1,đánh,V,O
...,...,...,...,...
314520,8847,cho,E,O
314521,8847,hướng,N,O
314522,8847,xử,N,O
314523,8847,trí,N,O


In [ ]:
df = df.fillna(method='ffill')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314525 entries, 0 to 314524
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   sentence_idx  314525 non-null  int64 
 1   word          314525 non-null  object
 2   pos           314525 non-null  object
 3   tag           314525 non-null  object
dtypes: int64(1), object(3)
memory usage: 9.6+ MB


In [ ]:
df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,314485,314486,314487,314488,314489,314490,314491,314492,314493,314494,314495,314496,314497,314498,314499,314500,314501,314502,314503,314504,314505,314506,314507,314508,314509,314510,314511,314512,314513,314514,314515,314516,314517,314518,314519,314520,314521,314522,314523,314524
sentence_idx,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,8846,8846,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847,8847
word,Tiếp,tục,xảy,ra,đánh,bom,nhằm,vào,xe,quân,sự,tại,Thái,Lan,Cảnh,sát,Thái,Lan,cho,biết,đã,có,4,binh,sỹ,của,quân,đội,nước,này,thiệt,mạng,và,6,người,khác,bị,thương,;,trong,...,),.,-,Khi,đang,dùng,thuốc,nếu,bị,dấu,hiệu,bất,thường,(,có,thể,bị,ADR,),nên,ngưng,ngay,thuốc,và,đi,tái,khám,báo,cho,bác,sĩ,biết,để,bác,sĩ,cho,hướng,xử,trí,.
pos,V,V,V,V,V,N,V,E,N,N,N,E,Np,Np,N,N,Np,Np,V,V,R,V,M,N,N,E,N,N,N,P,V,V,C,M,N,A,V,V,CH,E,...,CH,CH,CH,N,R,V,N,C,V,V,V,A,A,CH,R,R,V,Np,CH,C,V,T,N,C,V,V,V,N,E,N,N,V,E,V,V,E,N,N,N,CH
tag,O,O,O,O,O,O,O,O,O,O,O,O,B-LOCATION,I-LOCATION,O,O,B-LOCATION,I-LOCATION,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O


In [ ]:
df['sentence_idx'].nunique(), df.word.nunique(), df.pos.nunique(), df.tag.nunique()

(8847, 8912, 19, 9)

In [ ]:
df.tag.value_counts()

O                  282585
I-ORGANIZATION       9219
I-LOCATION           5056
B-LOCATION           4675
B-PERSON             3806
I-PERSON             3615
B-ORGANIZATION       3278
I-MISCELLANEOUS      1574
B-MISCELLANEOUS       717
Name: tag, dtype: int64

In [ ]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

In [ ]:
agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['word'].values.tolist(), 
                                                   s['pos'].values.tolist(), 
                                                   s['tag'].values.tolist())]

In [ ]:
grouped_df = df.groupby('sentence_idx').apply(agg_func)

In [ ]:
print(grouped_df[grouped_df.index == 1print(grouped_df[grouped_df.index == 'Sentence: 1'].values)].values)

[list([('Tiếp', 'V', 'O'), ('tục', 'V', 'O'), ('xảy', 'V', 'O'), ('ra', 'V', 'O'), ('đánh', 'V', 'O'), ('bom', 'N', 'O'), ('nhằm', 'V', 'O'), ('vào', 'E', 'O'), ('xe', 'N', 'O'), ('quân', 'N', 'O'), ('sự', 'N', 'O'), ('tại', 'E', 'O'), ('Thái', 'Np', 'B-LOCATION'), ('Lan', 'Np', 'I-LOCATION'), ('Cảnh', 'N', 'O'), ('sát', 'N', 'O'), ('Thái', 'Np', 'B-LOCATION'), ('Lan', 'Np', 'I-LOCATION'), ('cho', 'V', 'O'), ('biết', 'V', 'O'), ('đã', 'R', 'O'), ('có', 'V', 'O'), ('4', 'M', 'O'), ('binh', 'N', 'O'), ('sỹ', 'N', 'O'), ('của', 'E', 'O'), ('quân', 'N', 'O'), ('đội', 'N', 'O'), ('nước', 'N', 'O'), ('này', 'P', 'O'), ('thiệt', 'V', 'O'), ('mạng', 'V', 'O'), ('và', 'C', 'O'), ('6', 'M', 'O'), ('người', 'N', 'O'), ('khác', 'A', 'O'), ('bị', 'V', 'O'), ('thương', 'V', 'O'), (';', 'CH', 'O'), ('trong', 'E', 'O'), ('đó', 'P', 'O'), ('có', 'V', 'O'), ('\u200b', 'N', 'O'), ('một', 'M', 'O'), ('dân', 'N', 'O'), ('thường', 'N', 'O'), ('do', 'E', 'O'), ('vụ', 'N', 'O'), ('nổ', 'V', 'O'), ('bom', 'N',

In [ ]:
grouped_df.shape

(8847,)

In [ ]:
sentences = [s for s in grouped_df]
sentences[0]

[('Tiếp', 'V', 'O'),
 ('tục', 'V', 'O'),
 ('xảy', 'V', 'O'),
 ('ra', 'V', 'O'),
 ('đánh', 'V', 'O'),
 ('bom', 'N', 'O'),
 ('nhằm', 'V', 'O'),
 ('vào', 'E', 'O'),
 ('xe', 'N', 'O'),
 ('quân', 'N', 'O'),
 ('sự', 'N', 'O'),
 ('tại', 'E', 'O'),
 ('Thái', 'Np', 'B-LOCATION'),
 ('Lan', 'Np', 'I-LOCATION'),
 ('Cảnh', 'N', 'O'),
 ('sát', 'N', 'O'),
 ('Thái', 'Np', 'B-LOCATION'),
 ('Lan', 'Np', 'I-LOCATION'),
 ('cho', 'V', 'O'),
 ('biết', 'V', 'O'),
 ('đã', 'R', 'O'),
 ('có', 'V', 'O'),
 ('4', 'M', 'O'),
 ('binh', 'N', 'O'),
 ('sỹ', 'N', 'O'),
 ('của', 'E', 'O'),
 ('quân', 'N', 'O'),
 ('đội', 'N', 'O'),
 ('nước', 'N', 'O'),
 ('này', 'P', 'O'),
 ('thiệt', 'V', 'O'),
 ('mạng', 'V', 'O'),
 ('và', 'C', 'O'),
 ('6', 'M', 'O'),
 ('người', 'N', 'O'),
 ('khác', 'A', 'O'),
 ('bị', 'V', 'O'),
 ('thương', 'V', 'O'),
 (';', 'CH', 'O'),
 ('trong', 'E', 'O'),
 ('đó', 'P', 'O'),
 ('có', 'V', 'O'),
 ('\u200b', 'N', 'O'),
 ('một', 'M', 'O'),
 ('dân', 'N', 'O'),
 ('thường', 'N', 'O'),
 ('do', 'E', 'O'),
 ('vụ', 

In [ ]:
sent2features(sentences[0][5:7])

[{'+1:postag': 'V',
  '+1:postag[:2]': 'V',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:word.lower()': 'nhằm',
  'BOS': True,
  'bias': 1.0,
  'postag': 'N',
  'postag[:2]': 'N',
  'word.isdigit()': False,
  'word.istitle()': False,
  'word.isupper()': False,
  'word.lower()': 'bom',
  'word[-2:]': 'om',
  'word[-3:]': 'bom'},
 {'-1:postag': 'N',
  '-1:postag[:2]': 'N',
  '-1:word.istitle()': False,
  '-1:word.isupper()': False,
  '-1:word.lower()': 'bom',
  'EOS': True,
  'bias': 1.0,
  'postag': 'V',
  'postag[:2]': 'V',
  'word.isdigit()': False,
  'word.istitle()': False,
  'word.isupper()': False,
  'word.lower()': 'nhằm',
  'word[-2:]': 'ằm',
  'word[-3:]': 'hằm'}]

In [ ]:
sent2labels(sentences[0][5:7])

['O', 'O']

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

X = np.array([sent2features(s) for s in sentences])
y = np.array([sent2labels(s) for s in sentences])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train.shape, X_test.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """


((5750,), (3097,))

In [ ]:
!pip install sklearn-crfsuite

In [ ]:
import sklearn_crfsuite

crf = sklearn_crfsuite.CRF(algorithm='lbfgs',
                           c1=0.1,
                           c2=0.1,
                           max_iterations=100,
                           all_possible_transitions=True,
                           verbose=True)

In [ ]:
crf.fit(X_train, y_train)

loading training data to CRFsuite: 100%|██████████| 5750/5750 [00:03<00:00, 1797.10it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 41109
Seconds required: 0.525

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 400
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.56  loss=186472.98 active=40939 feature_norm=1.00
Iter 2   time=0.30  loss=120981.62 active=39303 feature_norm=3.05
Iter 3   time=0.30  loss=105284.36 active=38923 feature_norm=2.78
Iter 4   time=0.86  loss=77952.67 active=37524 feature_norm=2.31
Iter 5   time=0.29  loss=71990.87 active=39707 feature_norm=2.63
Iter 6   time=0.29  loss=62636.82 active=37208 feature_norm=3.35
Iter 7   time=0.29  loss=57583.75 active=33749 feature_norm=5.51
Iter 8   time=0.29  loss=49663.74 active=35686 feature_norm=5.15
Iter 9   time=0.30  loss=46704.90 active=36218 feature_norm=5.45
Iter 10  ti

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=400,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=True)

In [ ]:
print(X_test[0])

[{'bias': 1.0, 'word.lower()': 'bé', 'word[-3:]': 'Bé', 'word[-2:]': 'Bé', 'word.isupper()': False, 'word.istitle()': True, 'word.isdigit()': False, 'postag': 'N', 'postag[:2]': 'N', 'BOS': True, '+1:word.lower()': 'tên', '+1:word.istitle()': False, '+1:word.isupper()': False, '+1:postag': 'N', '+1:postag[:2]': 'N'}, {'bias': 1.0, 'word.lower()': 'tên', 'word[-3:]': 'tên', 'word[-2:]': 'ên', 'word.isupper()': False, 'word.istitle()': False, 'word.isdigit()': False, 'postag': 'N', 'postag[:2]': 'N', '-1:word.lower()': 'bé', '-1:word.istitle()': True, '-1:word.isupper()': False, '-1:postag': 'N', '-1:postag[:2]': 'N', '+1:word.lower()': 'lâm', '+1:word.istitle()': True, '+1:word.isupper()': False, '+1:postag': 'Np', '+1:postag[:2]': 'Np'}, {'bias': 1.0, 'word.lower()': 'lâm', 'word[-3:]': 'Lâm', 'word[-2:]': 'âm', 'word.isupper()': False, 'word.istitle()': True, 'word.isdigit()': False, 'postag': 'Np', 'postag[:2]': 'Np', '-1:word.lower()': 'tên', '-1:word.istitle()': False, '-1:word.isu

In [ ]:
y_pred = crf.predict(X_test)
print(y_pred[0])

['O', 'O', 'B-PERSON', 'I-PERSON', 'I-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O', 'O']


In [ ]:
from sklearn_crfsuite import metrics as crf_metrics

labels = list(crf.classes_)
labels.remove('O')

In [ ]:
print(crf_metrics.flat_classification_report(y_test, y_pred, labels=labels))

                 precision    recall  f1-score   support

       B-PERSON       0.87      0.85      0.86      1362
       I-PERSON       0.83      0.84      0.84      1293
     B-LOCATION       0.84      0.78      0.81      1618
     I-LOCATION       0.80      0.68      0.73      1750
 B-ORGANIZATION       0.81      0.69      0.75      1112
 I-ORGANIZATION       0.72      0.64      0.68      3116
B-MISCELLANEOUS       0.81      0.35      0.49       259
I-MISCELLANEOUS       0.74      0.36      0.48       549

      micro avg       0.80      0.70      0.75     11059
      macro avg       0.80      0.65      0.70     11059
   weighted avg       0.79      0.70      0.74     11059

